# Final Blending of the models

In [1]:
#best score so far
#alpha = 1.6 beta = 1. predict_proba, NB + log 88,94662%
#alpha = .3 beta = .2 predict_proba, NB + log 88,95450


##### check scores with train_test_split

In [2]:
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.svm import LinearSVC, LinearSVR, SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier, LinearRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import cross_val_score, train_test_split,StratifiedKFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.calibration import CalibratedClassifierCV
from lib_DSG import TransformerMixin, ColumnSelector


#confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score

folder = os.getcwd() ; print folder

/home/arda/Documents/DSG


In [3]:
#pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_tagged')
#pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_tagged')

pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_munged')
pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_munged')

##### Preprocessing: Logistic regression

In [4]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }

tfv_uni = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_lowercase_only"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_bi = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_word1"],
                ngram_range=(2, 2), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_nnp = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_NNP"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_punctuation = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_punct"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

log = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=2.0, fit_intercept=True, intercept_scaling=1, 
                   class_weight=None, random_state=None, solver='liblinear', max_iter=100,
                   multi_class='ovr', verbose=0)

uni = make_pipeline(ColumnSelector(key='Sentence'), tfv_uni)
bi = make_pipeline(ColumnSelector(key='Sentence'), tfv_bi)
nnp = make_pipeline(ColumnSelector(key='Sentence'), tfv_nnp)
punctuation = make_pipeline(ColumnSelector(key='Sentence'), tfv_punctuation)


log_pipe = make_union(uni, bi, nnp, punctuation)
log_pipe.transformer_weights=[1.2, 1, 1., 1]

##### Preprocessing: MultinomialNB

In [5]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }

tfv_uni = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_lowercase_only"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_bi = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_word1"],
                ngram_range=(2, 2), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_nnp = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_NNP"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_punctuation = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_punct"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

NB = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

uni = make_pipeline(ColumnSelector(key='Sentence'), tfv_uni)
bi = make_pipeline(ColumnSelector(key='Sentence'), tfv_bi)
nnp = make_pipeline(ColumnSelector(key='Sentence'), tfv_nnp)
punctuation = make_pipeline(ColumnSelector(key='Sentence'), tfv_punctuation)


NB_pipe = make_union(uni, bi, nnp, punctuation)
NB_pipe.transformer_weights=[1.3, 1, 1.1, 1]

##### Preprocessing: SVM

In [6]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }

tfv_uni = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_lowercase_only"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_bi = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_word1"],
                ngram_range=(2, 2), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_nnp = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_NNP"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_punctuation = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_punct"],
                ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

svm = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr',
          fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
          random_state=None, max_iter=1000)

uni = make_pipeline(ColumnSelector(key='Sentence'), tfv_uni)
bi = make_pipeline(ColumnSelector(key='Sentence'), tfv_bi)
nnp = make_pipeline(ColumnSelector(key='Sentence'), tfv_nnp)
punctuation = make_pipeline(ColumnSelector(key='Sentence'), tfv_punctuation)


svm_pipe = make_union(uni, bi, nnp, punctuation)
svm_pipe.transformer_weights=[1.2, 1, 1.1, 1]

In [7]:
Y = pd_train['Author'].values

# Testing each models

In [100]:
#X = sgd_pipe.fit_transform(pd_train)
#X = log_pipe.fit_transform(pd_train)
#X = NB_pipe.fit_transform(pd_train)
X = svm_pipe.fit_transform(pd_train)
#X = rf_pipe.fit_transform(pd_train)
#X = ridge_pipe.fit_transform(pd_train)

for cv in range(3):
    
    clf=svm
    
    skf = StratifiedKFold(Y, n_folds=5, indices=None, shuffle=True, random_state=None)
    scores_skf = cross_val_score(clf, X,Y,scoring='accuracy',cv=skf, n_jobs=-1)

    sss = StratifiedShuffleSplit(Y, 5, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X,Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SLF: acc: %0.4f, std: %0.4f, SSS: acc: %0.4f, std: %0.4f" %
           (scores_skf.mean(), scores_skf.std(), scores_sss.mean(), scores_sss.std()))

SLF: acc: 0.8017, std: 0.0058, SSS: acc: 0.8014, std: 0.0024
SLF: acc: 0.8006, std: 0.0020, SSS: acc: 0.8014, std: 0.0024
SLF: acc: 0.8016, std: 0.0041, SSS: acc: 0.8014, std: 0.0024


# BLENDING : meta classifier

In [91]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

In [27]:
x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2, random_state=345)

X_nb  = NB_pipe.fit_transform(x_train)
X_log = log_pipe.fit_transform(x_train)
#X_tag = tag_pipe.fit_transform(x_train)
#X_svm = svm_pipe.fit_transform(x_train)
X_rf = rf_pipe.fit_transform(x_train)



Xval_nb  = NB_pipe.transform(x_val)
Xval_log = log_pipe.transform(x_val)
#Xval_tag = tag_pipe.transform(x_val)
#Xval_svm = svm_pipe.transform(x_val)
Xval_rf = rf_pipe.transform(x_val)

#fitting on original data
NB.fit(X_nb, y_train)
log.fit(X_log, y_train)
#svm.fit(X_svm, y_train)
rf.fit(X_rf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=14, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
print X_svm.shape
print X_log.shape
print X_nb.shape

(22978, 95735)
(22978, 58411)
(22978, 57779)


In [28]:
metaXtrain = np.hstack((NB.predict_proba(X_nb),log.predict_proba(X_log), rf.predict_proba(X_rf)))

In [79]:
alphas = [0.1, 1., 10., 100.] 

alphas = 10.**-np.arange(1,7)
#alphas = [9e-3, 1e-4, 2e-4, 3e-4, 6e-4, 15e-4]
#alphas = [0.0006]
#alphas = np.arange(1, 5, 1)
alphas = [0.8e-3, 1e-3, 3e-3, 6e-3, 9e-3, 12e-3]

print alphas

[0.0008, 0.001, 0.003, 0.006, 0.009, 0.012]


In [70]:
LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [98]:
alpha = 8.5
beta = 8.0


for i in alphas:
    meta_log = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=i, fit_intercept=True, 
                                  intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear',
                                  max_iter=100, multi_class='ovr', verbose=0)
    
    meta_svm = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=i, multi_class='ovr',
                       fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                       random_state=None, max_iter=1000)
    
    
    
    clf = meta_svm
    
    
    x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2, random_state=np.random.randint(1000))

    X_nb  = NB_pipe.fit_transform(x_train)
    X_log = log_pipe.fit_transform(x_train)
    #X_tag = tag_pipe.fit_transform(x_train)
    X_svm = svm_pipe.fit_transform(x_train)
    #X_rf = rf_pipe.fit_transform(x_train)

    Xval_nb  = NB_pipe.transform(x_val)
    Xval_log = log_pipe.transform(x_val)
    #Xval_tag = tag_pipe.transform(x_val)
    Xval_svm = svm_pipe.transform(x_val)
    #Xval_rf = rf_pipe.transform(x_val)

    #fitting on original data
    NB.fit(X_nb, y_train)
    log.fit(X_log, y_train)
    svm.fit(X_svm, y_train)
    #rf.fit(X_rf, y_train)

    #building metaX
    metaXtrain = np.hstack((NB.predict_proba(X_nb),log.predict_proba(X_log)))
    #metaXtrain = np.hstack((NB.predict_proba(X_nb), log.predict_proba(X_log), rf.predict_proba(X_rf) ))
    metaY = y_train

    #fitting META clf

    clf.fit(metaXtrain, metaY)
    #meta_svm.fit(metaXtrain, metaY)

    #building eval data
    nb_pred = NB.predict_proba(Xval_nb)
    log_pred = log.predict_proba(Xval_log)
    #rf_pred = rf.predict_proba(Xval_rf)

    #metaXval = np.hstack((nb_pred,log_pred,rf_pred))
    metaXval = np.hstack((nb_pred,log_pred))
    
    y_pred_log = clf.predict(metaXval)
    #y_pred_svm = meta_svm.predict(metaXval)
    pred_normal_mean = clf.classes_[(( alpha*nb_pred + beta*log_pred) /2.).argmax(axis=1)]
    
    
    print accuracy_score(NB.predict(Xval_nb), y_val)
    print accuracy_score(log.predict(Xval_log), y_val)
    
    print accuracy_score(y_pred_log, y_val)
    #print accuracy_score(y_pred_svm, y_val)
    
    print accuracy_score(pred_normal_mean, y_val)
    print '\n'

0.807832898172
0.803307223673
0.812880765883
0.815491731941


0.804351610096
0.797389033943
0.81044386423
0.815317667537


0.798955613577
0.798955613577
0.805221932115
0.811314186249


0.802610966057
0.801914708442
0.805221932115
0.818450826806


0.812184508268
0.8
0.80591818973
0.821235857267


0.816187989556
0.805395996519
0.80818102698
0.826631853786




In [218]:
alpha = 8.5
beta = 8.0


for i in alphas:
    meta_log = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=i, fit_intercept=True, 
                                  intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear',
                                  max_iter=100, multi_class='ovr', verbose=0)
    
    meta_svm = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=i, multi_class='ovr',
                       fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                       random_state=None, max_iter=1000)
    
    
    
    clf = meta_svm
    
    
    x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2, random_state=np.random.randint(1000))

    X_nb  = NB_pipe.fit_transform(x_train)
    X_log = log_pipe.fit_transform(x_train)
    #X_tag = tag_pipe.fit_transform(x_train)
    X_svm = svm_pipe.fit_transform(x_train)
    #X_rf = rf_pipe.fit_transform(x_train)

    Xval_nb  = NB_pipe.transform(x_val)
    Xval_log = log_pipe.transform(x_val)
    #Xval_tag = tag_pipe.transform(x_val)
    Xval_svm = svm_pipe.transform(x_val)
    #Xval_rf = rf_pipe.transform(x_val)

    #fitting on original data
    NB.fit(X_nb, y_train)
    log.fit(X_log, y_train)
    svm.fit(X_svm, y_train)
    #rf.fit(X_rf, y_train)

    #building metaX
    dic = {'austen': 1, 'doyle': 2, 'poe': 3, 'shakespeare': 4, 'twain': 5, 'wilde': 6}
    dic1 = {1:'austen', 2:'doyle', 3:'poe', 4:'shakespeare', 5:'twain', 6:'wilde'}
    metaXtrain = pd.DataFrame(np.vstack((NB.predict(X_nb), log.predict(X_log), svm.predict(X_svm))).T).apply(lambda r: [dic[x] for x in r.values])
    metaY = y_train
    
    #fitting META clf
    clf.fit(metaXtrain, metaY)
    #meta_svm.fit(metaXtrain, metaY)
    
    #building eval data
    nb_pred = NB.predict(Xval_nb)
    log_pred = log.predict(Xval_log)
    #rf_pred = rf.predict_proba(Xval_rf)
    svm_pred = svm.predict(Xval_svm)

    #metaXval = np.hstack((nb_pred,log_pred,rf_pred))
    metaXval = pd.DataFrame(np.vstack((nb_pred, log_pred, svm_pred)).T).apply(lambda r: [dic[x] for x in r.values])
    
    y_pred = clf.predict(metaXval)
    #y_pred_svm = meta_svm.predict(metaXval)    
    y_pred = np.round(metaXval.mean(axis=1)).apply(lambda r: dic1[r])
    
    
    print accuracy_score(NB.predict(Xval_nb), y_val)
    print accuracy_score(log.predict(Xval_log), y_val)
    
    print accuracy_score(y_pred, y_val)
    
    print '\n'

0.805744125326
0.799303742385
0.756657963446


0.794255874674
0.79181897302
0.746910356832


0.807832898172
0.795648389904
0.755613577023


0.79773716275
0.791993037424
0.749869451697




KeyboardInterrupt: 

# BLENDING : Averaging

In [8]:
def getProbaSVM(dist_mat):

    dist_mat = dist_mat - dist_mat.min()

    dist_mat = dist_mat / dist_mat.max()

    return dist_mat

##### proba prediction

In [101]:
x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2, random_state=np.random.randint(1000))

X_nb  = NB_pipe.fit_transform(x_train)
X_log = log_pipe.fit_transform(x_train)
#X_tag = tag_pipe.fit_transform(x_train)
#X_tag = tag_pipe.fit_transform(x_train)
X_svm = svm_pipe.fit_transform(x_train)

Xval_nb  = NB_pipe.transform(x_val)
Xval_log = log_pipe.transform(x_val)
#Xval_tag = tag_pipe.transform(x_val)
Xval_svm = svm_pipe.transform(x_val)

nb_pred = NB.fit(X_nb,y_train).predict_proba(Xval_nb)
log_pred = log.fit(X_log,y_train).predict_proba(Xval_log)
#tag_pred = tag.fit(X_tag,y_train).predict_proba(Xval_tag)
mat_dist = svm.fit(X_svm, y_train).decision_function(Xval_svm)
svm_pred = getProbaSVM(mat_dist)

##### model 3 params: alpha, beta, gamma | 3models

In [35]:
for cv in range(5):
    
    x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2)

    X_nb  = NB_pipe.fit_transform(x_train)
    X_log = log_pipe.fit_transform(x_train)
    #X_tag = tag_pipe.fit_transform(x_train)
    #X_rf = rf_pipe.fit_transform(x_train)
    X_svm = svm_pipe.fit_transform(x_train)

    Xval_nb  = NB_pipe.transform(x_val)
    Xval_log = log_pipe.transform(x_val)
    #Xval_tag = tag_pipe.transform(x_val)
    #Xval_rf = rf_pipe.transform(x_val)
    Xval_svm = svm_pipe.transform(x_val)


    #nb_pred = NB.fit(X_nb,y_train).predict_proba(Xval_nb)
    #log_pred = log.fit(X_log,y_train).predict_proba(Xval_log)
    #tag_pred = tag.fit(X_tag,y_train).predict_proba(Xval_tag)
    #rf_pred = rf.fit(X_rf,y_train).predict_proba(Xval_rf)
    #mat_dist = svm.fit(X_svm, y_train).decision_function(Xval_svm)
    #svm_pred = getProbaSVM(mat_dist)    
    
    nb_pred = np.log(1 + NB.fit(X_nb,y_train).predict_proba(Xval_nb))
    log_pred = np.log(1 +log.fit(X_log,y_train).predict_proba(Xval_log))
    mat_dist = svm.fit(X_svm, y_train).decision_function(Xval_svm)
    svm_pred = np.log(1+getProbaSVM(mat_dist))
    
    results = []
    classes = NB.classes_

    for alpha in np.arange(0,1.5,.1):
        for beta in np.arange(0,1.5,.1):
            for gamma in np.arange(0,1.5,.1):

                pred_normal_mean = classes[( (alpha*nb_pred + beta*log_pred + gamma*svm_pred) ).argmax(axis=1)]
                pred_geo_mean = classes[(  nb_pred**(alpha) * log_pred**(beta) * svm_pred**(gamma)  ).argmax(axis=1)]

                #pred_normal_mean = classes[ (alpha*nb_pred + beta*log_pred).argmax(axis=1) ]
                #pred_geo_mean = classes[(  nb_pred**(alpha) * log_pred**(beta) ).argmax(axis=1)]
                
                score_normal_mean = accuracy_score(pred_normal_mean,y_val)
                score_geo_mean = accuracy_score(pred_geo_mean,y_val)

                results.append((alpha, beta, gamma, score_normal_mean, score_geo_mean))
                
    results = np.around(np.array(results),decimals=6)
    print "\n"
    print "score NB:", accuracy_score(classes[nb_pred.argmax(axis=1)],y_val)
    print "score log:", accuracy_score(classes[log_pred.argmax(axis=1)],y_val)
    print "score svm:", accuracy_score(classes[svm_pred.argmax(axis=1)],y_val)

    print "params normal mean", results[(results[:,3]).argmax()]
    print "params geo mean", results[(results[:,4]).argmax()]
    #print "params harmonic mean", results[(np.asarray(results)[:,4]).argmax()]




score NB: 0.810617928634
score log: 0.790948651001
score svm: 0.793385552654
params normal mean [ 0.2       0.1       0.8       0.822106  0.820191]
params geo mean [ 0.4       0.1       1.1       0.820191  0.821236]


score NB: 0.807832898172
score log: 0.793211488251
score svm: 0.78955613577
params normal mean [ 0.3       0.1       1.3       0.817058  0.815318]
params geo mean [ 0.2       0.1       0.6       0.816188  0.816188]


score NB: 0.818276762402
score log: 0.798607484769
score svm: 0.798781549173
params normal mean [ 0.6       0.2       1.1       0.823673  0.821584]
params geo mean [ 0.9       0.2       1.3       0.822628  0.823325]


KeyboardInterrupt: 

In [38]:
alpha = .2
beta = .1
gamma = 1.

for cv in range(4):
    x_train, x_val, y_train, y_val = train_test_split(pd_train,Y, test_size=0.2)    
    
    X_nb  = NB_pipe.fit_transform(x_train)
    X_log = log_pipe.fit_transform(x_train)
    #X_tag = tag_pipe.fit_transform(x_train)
    #X_rf = rf_pipe.fit_transform(x_train)
    X_svm = svm_pipe.fit_transform(x_train)

    Xval_nb  = NB_pipe.transform(x_val)
    Xval_log = log_pipe.transform(x_val)
    #Xval_tag = tag_pipe.transform(x_val)
    #Xval_rf = rf_pipe.transform(x_val)
    Xval_svm = svm_pipe.transform(x_val)

    #nb_pred = NB.fit(X_nb,y_train).predict_proba(Xval_nb)
    #log_pred = log.fit(X_log,y_train).predict_proba(Xval_log)
    #tag_pred = tag.fit(X_tag,y_train).predict_proba(Xval_tag)
    #rf_pred = rf.fit(X_rf,y_train).predict_proba(Xval_rf)
    #mat_dist = svm.fit(X_svm, y_train).decision_function(Xval_svm)
    #svm_pred = getProbaSVM(mat_dist)    
    
    nb_pred = np.log(1 + NB.fit(X_nb,y_train).predict_proba(Xval_nb))
    log_pred = np.log(1 +log.fit(X_log,y_train).predict_proba(Xval_log))
    mat_dist = svm.fit(X_svm, y_train).decision_function(Xval_svm)
    svm_pred = np.log(1 +getProbaSVM(mat_dist))
    
    
    pred_normal_mean = classes[( (alpha*nb_pred + beta*log_pred + gamma*svm_pred) ).argmax(axis=1)]
    pred_geo_mean = classes[(  nb_pred**(alpha) * log_pred**(beta) * svm_pred**(gamma)  ).argmax(axis=1)]

    #pred_normal_mean = classes[ (alpha*nb_pred + beta*log_pred).argmax(axis=1) ]
    #pred_geo_mean = classes[(  nb_pred**(alpha) * log_pred**(beta) ).argmax(axis=1)]
    
    score_normal_mean = accuracy_score(pred_normal_mean,y_val)
    score_geo_mean = accuracy_score(pred_geo_mean,y_val)

    
    print "score NB:", accuracy_score(classes[nb_pred.argmax(axis=1)],y_val)
    print "score svm:", accuracy_score(classes[svm_pred.argmax(axis=1)],y_val)
    
    print "score normal mean", score_normal_mean
    print "score geo mean", score_geo_mean
    #print "score harm mean", score_harm_mean
    print "diff :" , (score_normal_mean-accuracy_score(classes[nb_pred.argmax(axis=1)],y_val))
    print "diff :" , (score_geo_mean-accuracy_score(classes[nb_pred.argmax(axis=1)],y_val))
    


score NB: 0.810269799826
score svm: 0.795648389904
score normal mean 0.821758050479
score geo mean 0.820713664056
diff : 0.0114882506527
diff : 0.0104438642298
score NB: 0.813402959095
score svm: 0.801392515231
score normal mean 0.824891209748
score geo mean 0.824194952132
diff : 0.0114882506527
diff : 0.0107919930374
score NB: 0.813925152306
score svm: 0.791993037424
score normal mean 0.819321148825
score geo mean 0.819669277633
diff : 0.00539599651871
diff : 0.00574412532637
score NB: 0.817406440383
score svm: 0.792863359443
score normal mean 0.82454308094
score geo mean 0.822454308094
diff : 0.00713664055701
diff : 0.00504786771105


##### Kaggle final fitting on full dataset

In [39]:
alpha = .2
beta = .1
gamma = 1.

Y = pd_train['Author']

X_nb  = NB_pipe.fit_transform(pd_train)
X_log = log_pipe.fit_transform(pd_train)
#X_tag = tag_pipe.fit_transform(x_train)
#X_rf = rf_pipe.fit_transform(x_train)
X_svm = svm_pipe.fit_transform(pd_train)

Xval_nb  = NB_pipe.transform(pd_test)
Xval_log = log_pipe.transform(pd_test)
#Xval_tag = tag_pipe.transform(x_val)
#Xval_rf = rf_pipe.transform(x_val)
Xval_svm = svm_pipe.transform(pd_test)




nb_pred = np.log(1 + NB.fit(X_nb,Y).predict_proba(Xval_nb))
log_pred = np.log(1 +log.fit(X_log,Y).predict_proba(Xval_log))
mat_dist = svm.fit(X_svm, Y).decision_function(Xval_svm)
svm_pred = np.log(1 +getProbaSVM(mat_dist))


pred_normal_mean = classes[( (alpha*nb_pred + beta*log_pred + gamma*svm_pred) ).argmax(axis=1)]
#pred_normal_mean = classes[ (alpha*nb_pred + beta*svm_pred).argmax(axis=1) ]

In [40]:
y_pred = pred_normal_mean

# Final prediction majority vote

In [41]:
submit = pd.DataFrame(index=None)
submit['Id']=pd_test['Id']
submit['Pred']=np.asarray(y_pred)

In [42]:
y_pred

array(['wilde', 'doyle', 'austen', ..., 'doyle', 'twain', 'wilde'], 
      dtype='|S11')

In [43]:
submit.to_csv(folder+'/results/0.arda1.csv',sep=';',index=None)